# API esios de REE

El token de acceso es una clave personal que identifica tus peticiones a la API de ESIOS.
Sin él, la API rechaza cualquier petición.

La API espera que el token se proporcione en forma de x-api-key

La API se basa en indicadores, cada dato importante tiene un ID de indicador por ejemplo 1001

Valores de status code:
200	Datos correctos, 400 Parámetros incorrectos, 401 Token incorrecto, 403 Token sin permisos, 404 Indicador inexistente

In [8]:
import requests

TOKEN = "294da85af17aff04fda25ec930700cf59bde0ac3d8322d075617fc038c1ff224"

url = "https://api.esios.ree.es/indicators/1001"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    print("OK")
else:
    print("Errror code: %d", response.status_code)
print(response.json().keys())


OK
dict_keys(['indicator'])


### Ejemplo: obtener el precio horario del mercado eléctrico

Precio del mercado diario (POOL)

Indicador: 1001

In [12]:
import requests
import pandas as pd

TOKEN = "294da85af17aff04fda25ec930700cf59bde0ac3d8322d075617fc038c1ff224"

url = "https://api.esios.ree.es/indicators/1001"

params = {
    "start_date": "2024-01-01T00:00:00",  # Note: Changed lowercase 'z' to uppercase 'Z'
    "end_date": "2024-01-31T23:00:00",
    "time_trunc": "hour"
}

# CORRECT HEADER FOR ESIOS API
headers = {
    "x-api-key": TOKEN,  # Primary method: Use your token as the x-api-key
    # Alternatively, you can use the Authorization header in this specific format:
    # "Authorization": f'Token token="{TOKEN}"'  # Uncomment this line if 'x-api-key' doesn't work
}

try:
    response = requests.get(url, headers=headers, params=params)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        # Process your data here, for example into a DataFrame
        # df = pd.DataFrame(data['indicator']['values'])
        print("Request successful. Data keys:", data.keys())
    else:
        print("Request failed.")
        print("Response:", response.text)  # This may contain more specific error details

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Status Code: 200
Request successful. Data keys: dict_keys(['indicator'])


#### Problema de valores repetidos

Al considerarse varias zonas horarias, los datos se repiten. Solucion:
Agrupar por fecha-hora ignorando la zona horaria

In [ ]:
'''import requests
import pandas as pd

TOKEN = "294da85af17aff04fda25ec930700cf59bde0ac3d8322d075617fc038c1ff224"

url = "https://api.esios.ree.es/indicators/1001"

params = {
    "start_date": "2024-01-01T00:00:00Z",
    "end_date": "2024-01-31T23:00:00Z",
    "time_trunc": "hour"
}

headers = {"x-api-key": TOKEN}

response = requests.get(url, headers=headers, params=params)
data = response.json()  '''

values = data["indicator"]["values"]
df = pd.DataFrame(values)

print(f"Filas originales: {len(df)}")

# Convertir a datetime
df["datetime"] = pd.to_datetime(df["datetime"])

# Crear una columna con solo la fecha-hora sin zona horaria
df["datetime_simple"] = df["datetime"].dt.tz_localize(None)

# Agrupar por hora simple y tomar el primer valor
df_unique = df.groupby("datetime_simple", as_index=False).agg({
    "value": "first"  # Toma el primer valor de cada grupo
})

# Renombrar y ordenar
df_unique = df_unique.rename(columns={"datetime_simple": "datetime"})
df_unique = df_unique.sort_values("datetime")

print(f"Filas después de agrupar: {len(df_unique)}")
print(f"Horas esperadas (31 días): {31 * 24} = 744")
print(f"¿Tenemos datos únicos por hora?: {len(df_unique) == 744}")

# Mostrar las primeras 48 horas
print("\nPrimeras 48 horas únicas:")
print(df_unique.head(48).to_string(index=False))

Filas originales: 3720
Filas después de agrupar: 744
Horas esperadas (31 días): 744 = 744
¿Tenemos datos únicos por hora?: True

Primeras 48 horas únicas:
           datetime  value
2024-01-01 01:00:00 111.11
2024-01-01 02:00:00 110.77
2024-01-01 03:00:00 109.28
2024-01-01 04:00:00 110.16
2024-01-01 05:00:00 111.47
2024-01-01 06:00:00 109.95
2024-01-01 07:00:00 106.89
2024-01-01 08:00:00 107.31
2024-01-01 09:00:00 103.25
2024-01-01 10:00:00  76.42
2024-01-01 11:00:00  47.60
2024-01-01 12:00:00  46.40
2024-01-01 13:00:00  44.73
2024-01-01 14:00:00  44.17
2024-01-01 15:00:00  51.29
2024-01-01 16:00:00  68.26
2024-01-01 17:00:00  88.60
2024-01-01 18:00:00  96.15
2024-01-01 19:00:00 103.34
2024-01-01 20:00:00 110.10
2024-01-01 21:00:00 110.04
2024-01-01 22:00:00  94.09
2024-01-01 23:00:00  88.48
2024-01-02 00:00:00  91.46
2024-01-02 01:00:00  76.88
2024-01-02 02:00:00  56.30
2024-01-02 03:00:00  68.42
2024-01-02 04:00:00  63.93
2024-01-02 05:00:00  67.19
2024-01-02 06:00:00  89.71
2024-01-

### Convertir la respuesta a algo útil (DataFrame)

La API devuelve JSON complejo, pero lo importante está aqui

datetime → hora

value → precio €/MWh

In [ ]:
values = data["indicator"]["values"]

df = pd.DataFrame(values)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df[["datetime", "value"]]

print(df.head(48))


     value                       datetime          datetime_utc  \
0   123.83  2024-01-01T00:00:00.000+01:00  2023-12-31T23:00:00Z   
1   123.83  2024-01-01T00:00:00.000+01:00  2023-12-31T23:00:00Z   
2   123.83  2024-01-01T00:00:00.000+01:00  2023-12-31T23:00:00Z   
3   123.83  2024-01-01T00:00:00.000+01:00  2023-12-31T23:00:00Z   
4   123.83  2024-01-01T00:00:00.000+01:00  2023-12-31T23:00:00Z   
5   111.11  2024-01-01T01:00:00.000+01:00  2024-01-01T00:00:00Z   
6   111.11  2024-01-01T01:00:00.000+01:00  2024-01-01T00:00:00Z   
7   111.11  2024-01-01T01:00:00.000+01:00  2024-01-01T00:00:00Z   
8   111.11  2024-01-01T01:00:00.000+01:00  2024-01-01T00:00:00Z   
9   111.11  2024-01-01T01:00:00.000+01:00  2024-01-01T00:00:00Z   
10  110.77  2024-01-01T02:00:00.000+01:00  2024-01-01T01:00:00Z   
11  110.77  2024-01-01T02:00:00.000+01:00  2024-01-01T01:00:00Z   
12  110.77  2024-01-01T02:00:00.000+01:00  2024-01-01T01:00:00Z   
13  110.77  2024-01-01T02:00:00.000+01:00  2024-01-01T01:00:00

### Convertir Dataframe en un archivo CSV

Así evitamos tener que hacer un request de los datos todo el tiempo

In [ ]:
df_unique.to_csv("precio_pool_2024.csv", index=False)